# Capstone 2: West Nile Virus Prediction

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
train = pd.read_csv("train.csv")
weather = pd.read_csv("weather.csv")

Checking df size

In [3]:
train.shape

(10506, 12)

In [4]:
weather.shape

(2944, 22)

viewing data

In [5]:
train.head(3)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0


In [28]:
train.describe()

,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,35.687797,41.841139,-87.699908,7.819532,12.853512,0.052446
std,24.339468,0.112742,0.096514,1.452921,16.133816,0.222936
min,10.000000,41.644612,-87.930995,3.000000,1.000000,0.000000
25%,12.000000,41.732984,-87.760070,8.000000,2.000000,0.000000
50%,33.000000,41.846283,-87.694991,8.000000,5.000000,0.000000
75%,52.000000,41.954690,-87.627796,9.000000,17.000000,0.000000
max,98.000000,42.017430,-87.531635,9.000000,50.000000,1.000000


In [30]:
weather.describe()

,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.0,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,66.985054,0.477242,53.457880,59.228940,3.390625,5.622283,239.823370,923.173913,-0.500000,-1.0,-0.504042,0.022544,29.243281,29.873458,6.960666,17.494905,8.570686
std,0.500085,11.461970,10.381939,11.319494,5.057215,10.675181,9.542716,5.948150,6.107152,245.254649,926.377489,0.500085,0.0,0.500106,0.529213,1.126923,1.717239,3.587527,10.063609,3.160517
min,1.000000,41.000000,29.000000,-1.000000,-17.000000,22.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,0.100000,1.000000,-1.000000
25%,1.000000,69.000000,50.000000,60.000000,-1.000000,46.000000,53.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,0.000000,29.190000,29.870000,4.300000,7.000000,6.300000
50%,1.500000,78.000000,59.000000,69.000000,-1.000000,54.000000,61.000000,0.000000,4.000000,207.500000,823.000000,-0.500000,-1.0,-1.000000,0.000000,29.280000,29.970000,6.400000,19.000000,8.100000
75%,2.000000,85.000000,66.000000,75.000000,2.000000,62.000000,67.000000,5.000000,10.000000,446.250000,1859.250000,0.000000,-1.0,0.000000,0.060000,29.390000,30.060000,9.200000,25.000000,10.400000
max,2.000000,104.000000,83.000000,94.000000,23.000000,75.000000,78.000000,29.000000,29.000000,623.000000,1931.000000,0.000000,-1.0,0.100000,6.860000,29.860000,30.530000,24.100000,36.000000,26.300000


In [22]:
weather['Sunrise'].head(5)

Date
2007-05-01    448.0
2007-05-01     -1.0
2007-05-02    447.0
2007-05-02     -1.0
2007-05-03    446.0
Name: Sunrise, dtype: float64

Replacing alphabetic placeholders with numeric

In [7]:
weather = weather.replace('M', -1)
weather = weather.replace('-', -1)
weather = weather.replace('T', -1)
weather = weather.replace(' T', -1)
weather = weather.replace('  T', -1)


Verifying no null values in either train or weather.

In [8]:
pd.isna(train).sum()

Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64

In [9]:
pd.isna(weather).sum()

Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0
Water1         0
SnowFall       0
PrecipTotal    0
StnPressure    0
SeaLevel       0
ResultSpeed    0
ResultDir      0
AvgSpeed       0
dtype: int64

checking column data types

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10506 entries, 2007-05-29 to 2013-09-26
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Address                 10506 non-null  object 
 1   Species                 10506 non-null  object 
 2   Block                   10506 non-null  int64  
 3   Street                  10506 non-null  object 
 4   Trap                    10506 non-null  object 
 5   AddressNumberAndStreet  10506 non-null  object 
 6   Latitude                10506 non-null  float64
 7   Longitude               10506 non-null  float64
 8   AddressAccuracy         10506 non-null  int64  
 9   NumMosquitos            10506 non-null  int64  
 10  WnvPresent              10506 non-null  int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 984.9+ KB


In [16]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2944 entries, 2007-05-01 to 2014-10-31
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Tmax         2944 non-null   float64
 2   Tmin         2944 non-null   float64
 3   Tavg         2944 non-null   float64
 4   Depart       2944 non-null   float64
 5   DewPoint     2944 non-null   float64
 6   WetBulb      2944 non-null   float64
 7   Heat         2944 non-null   float64
 8   Cool         2944 non-null   float64
 9   Sunrise      2944 non-null   float64
 10  Sunset       2944 non-null   float64
 11  CodeSum      2944 non-null   object 
 12  Depth        2944 non-null   float64
 13  Water1       2944 non-null   float64
 14  SnowFall     2944 non-null   float64
 15  PrecipTotal  2944 non-null   float64
 16  StnPressure  2944 non-null   float64
 17  SeaLevel     2944 non-null   float64
 18  ResultSpeed  2944 non-null   f

Converting date columns to datetime object and setting the date as the index. 

In [12]:
train['Date'] = pd.to_datetime(train.Date, format='%Y-%m-%d')
train.set_index('Date', inplace=True, drop=True)
weather['Date'] = pd.to_datetime(weather.Date, format='%Y-%m-%d')
weather.set_index('Date', inplace=True, drop=True)

converting remaining columns in Weather with numeric values to int data types

**Rahul, do I need to convert the time-based columns? Not sure how to handle them and the '-1' null values. **

In [26]:
##Does 'train' columns that are the 'object' type need to be converted? 
for column in weather.columns[1:]:
        weather[column]=weather[column].astype(np.float64, errors='ignore')

weather.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2944 entries, 2007-05-01 to 2014-10-31
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Station      2944 non-null   int64  
 1   Tmax         2944 non-null   float64
 2   Tmin         2944 non-null   float64
 3   Tavg         2944 non-null   float64
 4   Depart       2944 non-null   float64
 5   DewPoint     2944 non-null   float64
 6   WetBulb      2944 non-null   float64
 7   Heat         2944 non-null   float64
 8   Cool         2944 non-null   float64
 9   Sunrise      2944 non-null   float64
 10  Sunset       2944 non-null   float64
 11  CodeSum      2944 non-null   object 
 12  Depth        2944 non-null   float64
 13  Water1       2944 non-null   float64
 14  SnowFall     2944 non-null   float64
 15  PrecipTotal  2944 non-null   float64
 16  StnPressure  2944 non-null   float64
 17  SeaLevel     2944 non-null   float64
 18  ResultSpeed  2944 non-null   f

In [14]:
weather.head(5)

,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,,,,,,
2007-05-01,1,83.0,50.0,67.0,14.0,51.0,56.0,0.0,2.0,448.0,...,,0.0,-1.0,0.0,0.0,29.10,29.82,1.7,27.0,9.2
2007-05-01,2,84.0,52.0,68.0,-1.0,51.0,57.0,0.0,3.0,-1.0,...,,-1.0,-1.0,-1.0,0.0,29.18,29.82,2.7,25.0,9.6
2007-05-02,1,59.0,42.0,51.0,-3.0,42.0,47.0,14.0,0.0,447.0,...,BR,0.0,-1.0,0.0,0.0,29.38,30.09,13.0,4.0,13.4
2007-05-02,2,60.0,43.0,52.0,-1.0,42.0,47.0,13.0,0.0,-1.0,...,BR HZ,-1.0,-1.0,-1.0,0.0,29.44,30.08,13.3,2.0,13.4
2007-05-03,1,66.0,46.0,56.0,2.0,40.0,48.0,9.0,0.0,446.0,...,,0.0,-1.0,0.0,0.0,29.39,30.12,11.7,7.0,11.9


In [38]:
merged = train.merge(weather, left_on='Date', right_on='Date')